In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import unidecode

In [ ]:
VOTES = pd.DataFrame(columns=['political_parties', 'total', 'share_valid', 'share_emited', 'Continent', 'Country', 'Estate', 'Ubigeo', 'Link'])

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
URL =  'https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E'

In [ ]:
def continente(driver):
    global VOTES

    continent = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select'))
    for continent_idx in range(len(continent.options)):
        continent = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select')).options[continent_idx]
        continent_name = continent.text

        if continent_name != "--TODOS--":
            continent.click()
            country = Select(driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select'))
            
            for country_idx in range( len(country.options) ):
                country = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
                country =  country.options[ country_idx ]
                
                country_name = country.text
            
                if country_name != "--TODOS--" :
                    country.click()
                    state = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select' ) )

                    for state_idx in range( len( state.options ) ):
                        state = Select( driver.find_element_by_xpath( '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select' ) )
                        state = state.options[ state_idx ]
                        
                        state_name = state.text
                        
                        if state_name != "--TODOS--":
                            state.click()
                            
                             ubigeo = driver.current_url.split("/")[-1]
                            # obtener tablas
                            table_path = driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[4]')
                            table_html = table_path.get_attribute('innerHTML')
                            # read table using pandas
                            table = pd.read_html(table_html)
                            table_clean = table[0].iloc[ 1:, 2: ].copy()
                            # new columns name
                            new_columns = table[0].iloc[0, 2:]
                            new_columns[0] = 'political_parties'
                            new_columns[1] = 'total'
                            new_columns[2] = 'share_valid'
                            new_columns[3] = 'share_emited'
                            # cleaning table
                            table_clean = table[0].iloc[ 1:, 2: ].copy()
                            # rename columns
                            table_clean.columns = new_columns
                            
                            # new values to columns
                            table_clean[ 'continent' ] = continent_name
                            table_clean[ 'country' ] = country_name
                            table_clean[ 'state' ] = state_name
                            table_clean[ 'ubigeo' ] = ubigeo
                            table_clean[ 'link' ] = driver.current_url
                            
                            # store tables
                            all_tables[ ubigeo ] = table_clean
                            
                            

In [ ]:
driver.get(URL)
continente(driver)